## Pretraining on unlabeled data

Thus far, we have implemented the data sampling and attention mechanism and
coded the LLM architecture. It is now time to implement a training function and
pretrain the LLM. We will learn about basic model evaluation techniques to mea-
sure the quality of the generated text, which is a requirement for optimizing the
LLM during the training process. 

In [1]:
import torch
from gptlight.config import GPTConfig, GPT2_CONFIG_124M
from gptlight.models import GPTModel
from gptlight.tokenizer import GPTTokenizer

from gptlight.utils import generate_text_simple

To order to reduce the computational demands of
training the model, making it possible to carry out the training on a standard laptop
computer, we shorten the context length from "vocab_size": 50257, 1,024 to 256 tokens.

In [22]:
GPT2_CONFIG_124M.context_length = 256
model = GPTModel(GPT2_CONFIG_124M)
tokenizer = GPTTokenizer()

### Evaluating generative text models

In [25]:
torch.manual_seed(123)
start_context = "Every effort moves you"

input_idx = tokenizer.encode(start_context)

token_ids = generate_text_simple(
    model=model,
    idx=input_idx,
    max_new_tokens=10,
    context_size=GPT2_CONFIG_124M.context_length
)

text_generated = tokenizer.decode(token_ids)
print(f"Output text :\n{text_generated}")

Output text :
Every effort moves you pregnant performTest DeV encouragement ther^^ dual Portugolics
